# Advent of Code 2024

In [43]:
import re
import numpy as np

## Day 1

In [2]:
with open('data/day1.txt') as f:
    lines = f.read().splitlines() 

# Format into two lists
list1 = []
list2 = []
for line in lines:
    a, b = line.split("   ")
    list1.append(int(a))
    list2.append(int(b))

# Sort lists in ascending order
list1 = sorted(list1)
list2 = sorted(list2)

distance = 0
for a, b in zip(list1, list2):
    distance += abs(a - b)

print(f"Part 1 answer is {distance}")

similarity = 0
for a in list1:
    # Count occurences of a in list2 then multiply by a
    similarity += a*len([i for i, x in enumerate(list2) if x == a])

print(f"Part 2 answer is {similarity}")

Part 1 answer is 1660292
Part 2 answer is 22776016


## Day 2

In [40]:
def is_safe(line):
    line = np.array([int(l) for l in line])
    diff = line[1:] - line[:-1]

    # disqualified if any diff is < 1 or > 3
    if (np.sum(np.abs(diff) > 3) == 0) and (np.sum(np.abs(diff) < 1) == 0):
        # disqualified if not all ascending or all descending
        if (np.sum(diff > 0) == 0) or (np.sum(diff < 0) == 0):
            return True
    else:
        return False

with open('data/day2.txt') as f:
    lines = f.read().splitlines()

lines = [l.split(" ") for l in lines]

# Part 1
safe = 0
for line in lines:
    if is_safe(line):
        safe += 1

print(f"Part 1 answer is {safe}")

# Part 2
safe = 0
for line in lines:
    for i in range(len(line)):
        # Just be lazy and make lots of copies of the list with a different item removed each time
        newline = line.copy()
        newline.pop(i)
        # If removing an item ever makes the list safe, it's safe
        if is_safe(newline):
            safe += 1
            break

print(f"Part 2 answer is {safe}")

Part 1 answer is 524
Part 2 answer is 569


## Day 3

In [87]:
with open('data/day3.txt') as f:
    line = f.read()

# Extract all strings matching the pattern + capture groups
patterns = re.findall(r"mul\((\d{1,3}),(\d{1,3})\)", line)

total = 0
for pattern in patterns:
    total += int(pattern[0])*int(pattern[1])

print(f"Part 1 answer is {total}")

# Part 2
patterns = re.findall(r"(mul\((\d{1,3}),(\d{1,3})\)|do\(\)|don't\(\))", line)

do = True
total = 0
for pattern in patterns:
    if pattern[0] == "don't()":
        do = False
    elif pattern[0] == "do()":
        do = True
    else:
        if do:
            total += int(pattern[1])*int(pattern[2])

print(f"Part 2 answer is {total}")

Part 1 answer is 170778545
Part 2 answer is 82868252


## Day 4

In [153]:
def count_string_in_array(line, string="XMAS"):
    # Turn it back into a string
    line = "".join(line)
    return len(re.findall(string, line))
    
with open('data/day4.txt') as f:
    lines = f.read().splitlines()

lines = [list(line) for line in lines]
lines = np.array(lines)

count = 0
# Find horizontal forwards and backwards
for line in lines:
    count += count_string_in_array(line)
    count += count_string_in_array(line[::-1])

# Vertical
for line in lines.transpose():
    count += count_string_in_array(line)
    count += count_string_in_array(line[::-1])

for i in range(-len(lines), len(lines)):
    # Diagonal one way
    line = lines.diagonal(i)
    count += count_string_in_array(line)
    count += count_string_in_array(line[::-1])
    # Diagonal the other way
    line = np.fliplr(lines).diagonal(i)
    count += count_string_in_array(line)
    count += count_string_in_array(line[::-1])

print(f"Part 1 answer is {count}")

# Find every 3x3 block
# Sliding filter convolution
count = 0
for i in range(lines.shape[0] - 2):
    for j in range(lines.shape[1] - 2):
        # Here is the 3 x 3 array
        grid = lines[i:i+3, j:j+3]
        diag1 = "".join(grid.diagonal())
        diag2 = "".join(np.fliplr(grid).diagonal())

        if (diag1 == "MAS") or (diag1 == "SAM"):
            if (diag2 == "MAS") or (diag2 == "SAM"):
                count += 1

print(f"Part 2 answer is {count}")

Part 1 answer is 2642
Part 2 answer is 1974
